In [1]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd, numpy as np
import os
from skimage import io
import torch
from torchvision import datasets, models, transforms
import torchvision
from skimage import color
from sklearn import metrics
import statistics

In [23]:
import matplotlib.pyplot as plt
import numpy as np
import time
import copy
from sklearn.metrics import mean_absolute_error
use_gpu = torch.cuda.is_available()

def train_model(model, criterion, optimizer, num_epochs=25, trainVal=['train','val'],verbose=True):
    since = time.time()
    
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    loss2plot = np.zeros([2,num_epochs])
    acc2plot  = np.zeros([2,num_epochs])
    for epoch in range(num_epochs):
        if verbose:
            print('Epoch {}/{}'.format(epoch, num_epochs - 1))
            print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in trainVal:
            if phase == 'train':
                imageLoader = train_loader
            else:
                imageLoader = validation_loader

            running_loss = 0.0
            running_total=0
            running_mad = []
            loss_list = []

            # Iterate over data.
            for idx, sample_batched in enumerate(imageLoader):
                # get the inputs
                inputs = sample_batched['x']
                peds_age = sample_batched['y']

                # wrap them in Variable
                if use_gpu:
                    inputs = inputs.type(torch.FloatTensor).cuda()
                    peds_age = peds_age.type(torch.FloatTensor).cuda()
                else:
                    inputs, peds_age = inputs.type(torch.FloatTensor), peds_age.type(torch.FloatTensor)
                
                peds_age = peds_age.view(-1,1)
                # zero the parameter gradients

                # forward
                outputs = model(inputs)
                #outputs = outputs.data
                loss = criterion(outputs,peds_age)
                optimizer.zero_grad()
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                
                num_imgs = inputs.size(0)
                running_loss += loss.item()*num_imgs
                running_total += num_imgs
                outputs= outputs.data.cpu().numpy()
                peds_age = peds_age.data.cpu().numpy()
                difference = mean_absolute_error(peds_age, outputs)
                running_mad.append(difference)
                loss_list.append(running_loss)
                
                # backward + optimize only if in training phase
            epoch_loss = running_loss / running_total
            epoch_acc = np.mean(running_mad)
            
            if verbose:
                print('{} Loss: {:.4f} MAE: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc))
            
            if phase == 'train':
                loss2plot[0,epoch] = epoch_loss
                acc2plot[0,epoch] = epoch_acc
            else:
                loss2plot[1,epoch] = epoch_loss
                acc2plot[1,epoch] = epoch_acc
                
                        # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())    
                
            #print('running mad', running_mad)    
        if verbose:
            print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    for phase in trainVal:
        if phase == 'train':
            idx=0
        else:
            idx=1
            
        fig = plt.figure()
        a = fig.add_subplot(2,2,2*idx+1)
        plt.plot(loss2plot[idx,:])
        plt.title('Loss per epoch for ' + phase)

        a = fig.add_subplot(2,2,2*idx+2)
        plt.plot(acc2plot[idx,:])
        plt.title('MAE per epoch for ' + phase)
        plt.show()
        
        
    plt.plot(loss2plot[0,:])
    plt.plot(loss2plot[1,:])
    plt.title('LOSS')
    plt.ylabel('Loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    plt.plot(acc2plot[0,:])
    plt.plot(acc2plot[1,:])
    plt.title('MAE')
    plt.ylabel('MAE')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model


In [24]:
model_resnet = torchvision.models.resnet34(pretrained=True)
for param in model_resnet.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = torch.nn.Linear(num_ftrs, 1)

In [25]:
train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([256,256]),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

validation_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([256,256]),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
class HXrayDataset_TL(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):

        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.data_frame.iloc[idx, 0])
        
        #some cases io.imread brings more channels than 1 due to bitsize issues
        image = io.imread(img_name)
        if len(image.shape) > 2 and image.shape[2] == 4:
            image = image[:,:,0]
            
        # replicate the image into 3 RGB channels
        image=np.repeat(image[None,...],3,axis=0)
            
        image_age = self.data_frame.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
            
        sample = {'x': image, 'y': image_age}

        return sample


In [26]:
BATCH_SIZE = 16

HXray_TrainData = HXrayDataset_TL(csv_file='data/train_boneage.csv',
                                    root_dir='/beegfs/ga4493/projects/team_G/boneage-training-dataset_final', transform=train_transform)
train_loader = DataLoader(HXray_TrainData, batch_size=BATCH_SIZE,
                        shuffle=True, num_workers=0)

HXray_ValidationData = HXrayDataset_TL(csv_file='data/validation_boneage.csv',
                                    root_dir='/beegfs/ga4493/projects/team_G/boneage-training-dataset_final', transform=validation_transform)
validation_loader = DataLoader(HXray_ValidationData, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=0)

dataset_sizes = {'train': len(HXray_TrainData), 'val': len(HXray_ValidationData)}


In [27]:
if use_gpu:
    model = model_resnet.cuda()
     #This will optimize only the final layer since other layers the gradient calculation is removed and only parameters from the fc layer is given to the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

criterion = torch.nn.MSELoss()

  

In [28]:
model_resnet.train()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [ ]:
model_resnet_ft = train_model(model_resnet, criterion, optimizer,
                       num_epochs=40, trainVal=['train','val'])

Epoch 0/39
----------
train Loss: 1840.6314 MAE: 34.9177
val Loss: 1924.0617 MAE: 35.3332

Epoch 1/39
----------
train Loss: 1793.1414 MAE: 34.4282
val Loss: 1889.0610 MAE: 35.1053

Epoch 2/39
----------
train Loss: 1769.6779 MAE: 34.1380
val Loss: 1864.8663 MAE: 34.9628

Epoch 3/39
----------
train Loss: 1746.6530 MAE: 33.9188
val Loss: 1849.4015 MAE: 34.7360

Epoch 4/39
----------
train Loss: 1728.3459 MAE: 33.7578
val Loss: 1847.6862 MAE: 34.4602

Epoch 5/39
----------
train Loss: 1717.4634 MAE: 33.6363
val Loss: 1827.0202 MAE: 34.6014

Epoch 6/39
----------
train Loss: 1719.5080 MAE: 33.6642
val Loss: 1833.7674 MAE: 34.2079

Epoch 7/39
----------
train Loss: 1705.1560 MAE: 33.5702
val Loss: 1819.9638 MAE: 34.2418

Epoch 8/39
----------
train Loss: 1695.6607 MAE: 33.4089
val Loss: 1814.9084 MAE: 34.1509

Epoch 9/39
----------
train Loss: 1689.3795 MAE: 33.3707
val Loss: 1806.6882 MAE: 34.2189

Epoch 10/39
----------
train Loss: 1675.2778 MAE: 33.2583
val Loss: 1807.3195 MAE: 34.1216